## ***Library***

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from skimage.feature import graycomatrix, graycoprops
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Part 1 : Binary classification**

# **Preparation of the dataset**

In [ ]:

def load_images_from_folder(folder):
    images = []
    for file in os.listdir(folder):
        if file.endswith(('.jpg', '.png')):
            img = cv2.imread(os.path.join(folder, file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir de BGR à RGB
            images.append(img)
    return np.array(images)

np.save("image_ernest.npy", load_images_from_folder("/content/drive/MyDrive/tp02apprentissageautomatique/ernest_celestine_01/ernest_celestine_01"))
np.save("images_toy_story.npy", load_images_from_folder("/content/drive/MyDrive/tp02apprentissageautomatique/toy_story_1_01/toy_story_1_01"))

In [ ]:
# Load the images into numpy arrays
x_ernest_celestine = load_images_from_folder("/content/drive/MyDrive/tp02apprentissageautomatique/ernest_celestine_01/ernest_celestine_01")
x_toy_story        = load_images_from_folder("/content/drive/MyDrive/tp02apprentissageautomatique/toy_story_1_01/toy_story_1_01")

print(x_ernest_celestine.shape)
print(x_toy_story.shape)

# Save the arrays
np.save("images_ernest.npy", x_ernest_celestine)
np.save("images_toy_story.npy", x_toy_story)

(500, 540, 920, 3)
(500, 540, 920, 3)


In [ ]:
## Function to extract simple features from an image
def extract_simple_features(image):

  mean_red   = np.mean(image[:,:,0])  # Calculate the mean intensity for the Red channel
  mean_green = np.mean(image[:,:,1])
  mean_blue  = np.mean(image[:,:,2])

 # Create the feature vector using the calculated means
  features_simple = np.array([mean_red, mean_green, mean_blue])

  return features_simple

image = cv2.imread('/content/drive/MyDrive/tp02apprentissageautomatique/ernest_celestine_01/ernest_celestine_01/ernest-celestine-00034.png')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
features_simple = extract_simple_features(image_rgb)
print("Feature vector:", features_simple)

x_ernest_features_simple    = np.array([extract_simple_features(image) for image in x_ernest_celestine])
x_toy_story_features_simple = np.array([extract_simple_features(image) for image in x_toy_story])

Feature vector: [101.70358696  87.4992351   72.34232085]


In [ ]:
# Assign labels: 0 for Ernest & Celestine, 1 for Toy Story
y_ernest_celestine_simple = np.zeros(x_ernest_features_simple.shape[0])
y_toy_story_simple        = np.ones(x_toy_story_features_simple.shape[0])


# Combine features and labels from both datasets
x_simple = np.concatenate([x_ernest_features_simple, x_toy_story_features_simple], axis=0)
y_simple = np.concatenate([y_ernest_celestine_simple, y_toy_story_simple], axis=0)

# Split the data into training and testing sets
x_train_simple , x_test_simple , y_train_simple , y_test_simple = train_test_split(x_simple , y_simple , test_size=0.2 , random_state= 42 )

# Flatten the feature data (540x920x3)
x_train_flat_simple = x_train_simple.reshape(x_train_simple.shape[0], -1)
x_test_flat_simple=  x_test_simple.reshape(x_test_simple.shape[0], -1)

print("Forme de x_train_flat:", x_train_flat_simple.shape)
print("Forme de x_test_flat:", x_test_flat_simple.shape)


Forme de x_train_flat: (800, 3)
Forme de x_test_flat: (200, 3)


In [ ]:
# Complex features extraction
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def extract_features(image):

    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)


    mean = np.mean(image, axis=(0, 1))
    variance = np.var(image, axis=(0, 1))


    hist = [np.histogram(image[:, :, i], bins=16, range=(0, 256))[0] for i in range(3)]


    glcm = graycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True)
    texture = [graycoprops(glcm, prop)[0, 0] for prop in ['contrast', 'homogeneity', 'energy', 'correlation']]

    features = np.hstack([mean, variance, *hist, texture])

    return features

image = cv2.imread('/content/drive/MyDrive/tp02apprentissageautomatique/ernest_celestine_01/ernest_celestine_01/ernest-celestine-00034.png')
#image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
features = extract_features(image_rgb)
#print("Feature vector:", features)
x_ernest_features    = np.array([extract_features(image) for image in x_ernest_celestine])
x_toy_story_features = np.array([extract_features(image) for image in x_toy_story])

In [ ]:
# Assign labels: 0 for Ernest & Celestine, 1 for Toy Story
y_ernest_celestine = np.zeros(x_ernest_features.shape[0])
y_toy_story        = np.ones(x_toy_story_features.shape[0])


# Combine features and labels from both datasets
x = np.concatenate([x_ernest_features, x_toy_story_features], axis=0)
y = np.concatenate([y_ernest_celestine, y_toy_story], axis=0)

# Split the data into training and testing sets
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2 , random_state= 42 )

# Flatten the feature data (540x920x3)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat =  x_test.reshape(x_test.shape[0], -1)




## **Model creation**

In [ ]:
model = models.Sequential([
    layers.Input(shape=(x_train_flat_simple.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()


history_simple = model.fit(x_train_flat_simple, y_train_simple, epochs=50, batch_size=16, validation_data=(x_test_flat_simple, y_test_simple))



Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_109 (Dense)                    │ (None, 16)                  │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_110 (Dense)                    │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_111 (Dense)                    │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 353 (1.38 KB)

 Trainable params: 353 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5525 - loss: 1.4695 - val_accuracy: 0.6050 - val_loss: 0.8133
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6204 - loss: 0.7570 - val_accuracy: 0.6100 - val_loss: 0.9225
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6653 - loss: 0.6544 - val_accuracy: 0.6050 - val_loss: 0.7504
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6334 - loss: 0.6234 - val_accuracy: 0.6200 - val_loss: 0.6840
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6200 - loss: 0.6541 - val_accuracy: 0.6150 - val_loss: 0.6565
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6338 - loss: 0.6136 - val_accuracy: 0.6650 - val_loss: 0.7669
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6955 - loss: 0.6572 - val_accuracy: 0.6250 - val_loss: 0.6106
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6796 - loss: 0.5440 - val_accuracy: 0.6400 - val_loss:

In [ ]:
# Evaluation of the model
loss, accuracy = model.evaluate(x_test_flat_simple, y_test_simple)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7630 - loss: 0.5196 
Test Accuracy: 77.50%


In [ ]:

model = models.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),  #
    layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()


Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_103 (Dense)                    │ (None, 16)                  │             944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_104 (Dense)                    │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_105 (Dense)                    │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,233 (4.82 KB)

 Trainable params: 1,233 (4.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

## Entraîner le modèle
history = model.fit(x_train_flat, y_train, epochs=50, batch_size=16, validation_data=(x_test_flat, y_test))

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9645 - loss: 29.3039 - val_accuracy: 0.8950 - val_loss: 143.7779
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9572 - loss: 38.1593 - val_accuracy: 0.9450 - val_loss: 110.1766
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9501 - loss: 31.6581 - val_accuracy: 0.9200 - val_loss: 144.9718
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9542 - loss: 38.6512 - val_accuracy: 0.9250 - val_loss: 120.3061
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9495 - loss: 24.1959 - val_accuracy: 0.9200 - val_loss: 136.5944
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9545 - loss: 34.0947 - val_accuracy: 0.9400 - val_loss: 119.8711
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9684 - loss: 12.3870 - val_accuracy: 0.9350 - val_loss: 120.5546
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9691 - loss: 13.5373 - val_accura

In [ ]:

loss, accuracy = model.evaluate(x_test_flat, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9249 - loss: 102.7745
Test Accuracy: 93.00%


4
